In [1]:
import warnings

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPool2D
from keras.models import Sequential
from keras.optimizers import RMSprop

seed = 42

import warnings
warnings.filterwarnings("ignore")

2025-09-21 21:35:58.940296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758490559.197681      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758490559.278240      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
root = '/kaggle/input/digit-recognizer/'
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
train_df = pd.read_csv(root + 'train.csv')
test_df = pd.read_csv(root + 'test.csv')

In [4]:
X = train_df.iloc[:, 1:]
y = train_df.iloc[:, 0]
X.iloc[0][133]

255

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = seed
)

X_train = X_train.values.reshape(-1, 28, 28) / 255.0
X_test = X_test.values.reshape(-1, 28, 28) / 255.0

y_train = y_train.values
X_train = np.expand_dims(X_train, axis = -1)
y_test = y_test.values

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 13,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = False,
    vertical_flip = False,
)

train_gen = datagen.flow(X_train, y_train)

In [7]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides = 1, activation = keras.activations.relu, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.strides = strides
        self.activation = activation
        self.main_layer = [
            keras.layers.Conv2D(
                filters, 3, strides = strides, padding = "same", use_bias = False
            ),
            keras.layers.BatchNormalization(),
            keras.layers.Activation("relu"),
            keras.layers.Conv2D(filters, 3, strides = 1, padding = "same", use_bias = False),
            keras.layers.BatchNormalization(),
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(
                    filters, 1, strides = strides, padding = "same", use_bias = False
                ),
                keras.layers.BatchNormalization(),
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layer:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [8]:
model = Sequential()

model.add(
    Conv2D(
        filters = 128,
        kernel_size = (3, 3),
        padding = "Same",
        activation = "relu",
        input_shape = (28, 28, 1),
    )
)
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(
    Conv2D(
        filters = 128,
        kernel_size = (3, 3),
        padding = "Same",
        activation = "relu",
        input_shape = (28, 28, 1),
    )
)
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5, 5), padding = "Same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (5, 5), padding = "Same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding = "Same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(BatchNormalization())
model.add(Dropout(0.20))

model.add(Dense(10, activation = "softmax"))

I0000 00:00:1758490578.345377      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758490578.345999      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [9]:
optimizer = RMSprop(learning_rate = 0.001, rho = 0.9, epsilon = 1e-08, decay = 0.0)
model.compile(
    optimizer = optimizer, loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)
early_stopping = keras.callbacks.EarlyStopping(patience = 12)

In [10]:
scheduler = ReduceLROnPlateau(
    monitor = "val_acc", patience = 3, verbose = 1, factor = 0.5, min_lr = 0.00001
)

In [11]:
history = model.fit(
    X_train,
    y_train,
    epochs = 200,
    validation_data = (X_test, y_test),
    callbacks = [early_stopping, scheduler],
)

Epoch 1/200


I0000 00:00:1758490584.358685      63 service.cc:148] XLA service 0x7a0e740080c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758490584.359621      63 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1758490584.359648      63 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1758490584.861934      63 cuda_dnn.cc:529] Loaded cuDNN version 90300


  14/1050 ━━━━━━━━━━━━━━━━━━━━ 13s 13ms/step - accuracy: 0.1676 - loss: 2.7698

I0000 00:00:1758490591.554575      63 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - accuracy: 0.8186 - loss: 0.5739 - val_accuracy: 0.9820 - val_loss: 0.0641 - learning_rate: 0.0010
Epoch 2/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9730 - loss: 0.0873 - val_accuracy: 0.9865 - val_loss: 0.0435 - learning_rate: 0.0010
Epoch 3/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9807 - loss: 0.0688 - val_accuracy: 0.9877 - val_loss: 0.0381 - learning_rate: 0.0010
Epoch 4/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9848 - loss: 0.0556 - val_accuracy: 0.9898 - val_loss: 0.0368 - learning_rate: 0.0010
Epoch 5/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9882 - loss: 0.0421 - val_accuracy: 0.9876 - val_loss: 0.0486 - learning_rate: 0.0010
Epoch 6/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9901 - loss: 0.0382 - val_accuracy: 0.9925 - val_loss: 0.0303 - learning_rate: 0.0010
Epoch 7/200
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0

In [12]:
test_df = test_df.values.reshape(-1, 28, 28) / 255.0
predictions = model.predict(test_df)
predicted_labels = np.argmax(predictions, axis = 1)
print(predicted_labels)

875/875 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
[2 0 9 ... 3 9 2]


In [13]:
submission = pd.DataFrame(
    {"ImageId": np.arange(1, len(predictions) + 1), "Label": predicted_labels}
)

print("Saving submission.csv...")
submission.to_csv("submission.csv", index = False)
print("submission.csv saved!")

Saving submission.csv...
submission.csv saved!
